# PhilReinholdPygrape tutorial 4: Two cavities one qubit

This notebook is based on the original examples written by Phil,
they can be found in [this directory](https://github.com/tesla-cat/yarn/tree/master/examplesFromForkedLibraries/PhilReinholdPygrape).
Specifically, it contains the following: 

- `make_fock_3mode.py`

**Note**: discard as it takes too long 

Ruiqi, 28 Jun 2020

In [1]:
import numpy as np
from qutip import *
from yarn.PhilReinholdPygrape import *
from yarn.qutipHelpers import (
    plotWignersIntermediateStates, 
    plotOccupationsIntermediateStates,
    plotExpectation, cat
)

## parameters

In [2]:
driveQ  = 2*np.pi * 0.196 # GHz
driveC1 = 2*np.pi * 0.279 # GHz
driveC2 = 2*np.pi * 0.158 # GHz

chiQC1 = -1.08e-3
chiQC2 = -1.04e-3

anharmonicityQ = -.1165 # GHz
kerrC1 = -2e-6 # GHz
kerrC2 = -2e-6 # GHz

## define the penalties

In [3]:
penalties = [
    make_amp_cost(1e-4, 0.02),
    make_lin_deriv_cost(1e-2)
]

## define the grape setup

In [4]:
def makeSetup(NC, NQ):
    aC1 = tensor(destroy(NC), qeye(NC), qeye(NQ))
    aC2 = tensor(qeye(NC), destroy(NC), qeye(NQ))
    aQ  = tensor(qeye(NC), qeye(NC), destroy(NQ))
    aC1d = aC1.dag(); aC2d = aC2.dag(); aQd = aQ.dag()
    
    Hdrift = chiQC1 * aC1d * aC1 * aQd * aQ
    Hdrift+= chiQC2 * aC2d * aC2 * aQd * aQ
    Hdrift+= (anharmonicityQ/2) * aQd * aQd * aQ * aQ
    Hdrift+= (kerrC1/2) * aC1d * aC1d * aC1 * aC1
    Hdrift+= (kerrC2/2) * aC2d * aC2d * aC2 * aC2
    Hdrift*= 2*np.pi
    
    HcontrolList = [
        driveQ  * (aQ  + aQd ), 1j*driveQ  * (aQ  - aQd ), 
        driveC1 * (aC1 + aC1d), 1j*driveC1 * (aC1 - aC1d), 
        driveC2 * (aC2 + aC2d), 1j*driveC2 * (aC2 - aC2d),
    ]

    inits = [tensor(fock(NC, 0), fock(NC, 0), fock(NQ, 0))]
    finals= [tensor(fock(NC, 1), fock(NC, 1), fock(NQ, 0))]

    setup = StateTransferSetup(
        Hdrift, HcontrolList, inits, finals, 
        use_taylor=True, sparse=True,
    )
    setup.taylor_order = 12
    return setup

ctrlNames = ['XQ','YQ', 'XC1','YC1', 'XC2','YC2']

setups = [makeSetup(8,3), makeSetup(9,3)]


## define the initial control sequence

In [5]:
initControls=5e-4*random_waves(len(ctrlNames), 1000, npoints=20)

## run grape

In [6]:
import warnings; warnings.filterwarnings('ignore')

result = run_grape(
    initControls, setups, 
    penalty_fns=penalties, 
    discrepancy_penalty=1e6, n_proc=2
)

0 - Fids: 0.005579122 0.005579122 Penalties: 5.51242e-05 6.348387e-09 3.761582e-30
1 - Fids: 0.005579122 0.005579122 Penalties: 5.51242e-05 6.348387e-09 3.761582e-30
2 - Fids: 0.2679725 0.03417865 Penalties: 0.1866756 7.052157e-07 27329.79
3 - Fids: 0.1759155 0.034593 Penalties: 0.01423265 8.511015e-08 9986.019
4 - Fids: 0.1938425 0.2179996 Penalties: 0.001559381 1.569127e-08 291.7829
5 - Fids: 0.2002082 0.2002021 Penalties: 0.0002290611 7.648676e-09 1.855077e-05
6 - Fids: 0.1456598 0.1456598 Penalties: 8.374583e-05 6.631393e-09 2.254252e-16
7 - Fids: 0.2401025 0.2401026 Penalties: 0.0001593437 7.181512e-09 4.239446e-09
8 - Fids: 0.1998537 0.1998278 Penalties: 0.0003151043 8.123773e-09 0.0003341762
9 - Fids: 0.2539718 0.2539717 Penalties: 0.0001974722 7.416237e-09 3.553792e-09
10 - Fids: 0.3641016 0.3641016 Penalties: 0.0002040822 7.28736e-09 5.982108e-13
11 - Fids: 0.3671526 0.3671526 Penalties: 0.0002251128 7.410202e-09 2.464224e-15
12 - Fids: 0.3675362 0.3675362 Penalties: 0.0002261

KeyboardInterrupt: 